In [1]:
# Import libraries  
import numpy as np
import pandas as pd
from time import time

In [2]:
# 1. (DNA)
X1 = 'ACTACTAGATTACTTACGGATCAGGTACTTTAGAGGCTTGCAACCA'
Y1 = 'TACTAGCTTACTTACCCATCAGGTTTTAGAGATGGCAACCA'

In [3]:
# 2. (Proteins)
X2 = 'AASRPRSGVPAQSDSDPCQNLAATPIPSRPPSSQSCQKCRADARQGRWGP'
Y2 = 'SGAPGQRGEPGPQGHAGAPGPPGPPGSDG'

### 1. Design an algorithm that, given strings X and Y , computes the edit distance between X and Y . The algorithm should provide also the optimal sequence of operations transforming X into Y .

In [4]:
# Using matrix dynamic programming, define a function which given string 1 and string 2, we calculate the edit distance

def edit_distance(string1, string2):
    """ Construct the initial matrix """
    m = len(string1) + 1 #length of string 1
    n = len(string2) + 1 #length of string 2
    matrix = np.zeros((m, n)) #create a matrix of dimensions m x n with zeros
    for x in range(m): #iterate over range of m
        matrix[x, 0] = x #put the values of the iteration in the first column
    for y in range(n): #iterate over range of n
        matrix[0, y] = y #put the values of the iteration in the first row
    
    """ Fill in the matrix with minimum cost """
    for x in range(1, m): #iterate over range of m skipping the first value [0]
        for y in range(1, n): #iterate over range of n skipping the first value [0]
            sub = 1 if string1[x-1] != string2[y-1] else 0 #determine if there was a substitution   
            matrix[x,y] = min( #select the action with minimum cost 
                matrix[x-1, y-1] + sub, #calculate the substitution cost
                matrix[x-1, y] + 1, #calculate the deletion cost
                matrix[x, y-1] + 1 #calculate the insertion cost
            )     
    
    """ Calculate the optimal sequence of operations """
    row = m-1 #create row variable
    column = n-1 #create column variable
    operations = [] #create list for optimal sequence of operations
    while not ((row == 0) and (column == 0)): #find optimal path from cell [row,column] until cell [0,0]
        if (row != 0) & (column != 0): #if we haven't reached the first row or the first column
            #Value of cell to the left is smaller than cell above and diagonal cell """
            if (matrix[row,column-1] < matrix[row-1,column]) & (matrix[row,column-1] < matrix[row-1,column-1]):
                column = column-1 #shift to the left
                operations.append('I') #add Insertion to our list 
            #Value of cell above is smaller than cell to the left and diagonal cell
            elif (matrix[row-1,column] < matrix[row,column-1]) & (matrix[row-1,column] < matrix[row-1,column-1]): 
                row = row-1 #shift upwards
                operations.append('D') #add Deletion to our list
            #Value of diagonal cell is smaller than cell to the left and cell above
            elif (matrix[row-1,column-1] <= matrix[row,column-1]) & (matrix[row-1,column-1] <= matrix[row-1,column]): 
                if matrix[row-1,column-1] == matrix[row,column] - 1: #value of diagonal cell is equal to current cell +1
                    row = row-1 #shift upwards
                    column = column-1 #and shift to the left 
                    operations.append('S') #add Substitution to our list
                else: #value of diagonal cell is equal to current cell
                    row = row-1 #shift upwards
                    column = column-1 #and shift to the left
                    operations.append('-') #add Blank as no operations were needed
        elif column != 0: #if we have reached to the first row
            column = column-1 #shift to the left
            operations.append('I') #add Insertion to our list
        else: #if we have reached to the first column
            row = row-1 #shift upwards
            operations.append('D') #add Deletion to our list
    operations = list(reversed(operations)) #reverse the order of the list, so that we have from beginning to end
    
    """ Print the results """
    print ('Optimal sequence: ', operations) #print the contructed matrix
    return (int(matrix[m - 1, n - 1])) #return the edit distance

In [5]:
# Print the results from DNA
st = time()
print('Edit distance: ', edit_distance(X1, Y1))

# Print time from DNA
print('Time: %0.5f seconds' % (time() - st))

Optimal sequence:  ['D', 'D', '-', '-', '-', '-', '-', '-', 'S', '-', '-', '-', '-', '-', '-', '-', '-', 'S', 'S', '-', '-', '-', '-', '-', '-', '-', 'D', 'D', '-', '-', '-', '-', '-', '-', '-', 'D', 'S', '-', 'S', '-', '-', '-', '-', '-', '-', '-']
Edit distance:  10
Time: 0.00398 seconds


In [6]:
# Print the results from Proteins
st = time()
print('Edit distance: ', edit_distance(X2, Y2))

# Print time from Proteins
print('Time: %0.5f seconds' % (time() - st))

Optimal sequence:  ['D', 'D', '-', 'D', 'D', 'D', 'D', '-', 'S', '-', 'S', '-', 'S', 'S', 'S', 'S', 'S', 'S', '-', 'S', 'S', '-', 'S', 'S', '-', 'S', '-', 'S', 'S', '-', '-', 'S', '-', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', '-', 'D', 'D', 'D', '-', 'D', 'D', 'D', 'D']
Edit distance:  37
Time: 0.00298 seconds


### 2. Modify the previous algorithm with a penalty function: operations D and I have unit cost 2, whereas operation S has unit cost 1.

In [7]:
# Using matrix dynamic programming, define a function which given string 1, string 2, 
# the substitution cost, the deletion cost and the insertion cost, we calculate the edit distance

def edit_distance_penalty(string1, string2, sub_cost=1, del_cost=2, ins_cost=2):
    """ Construct the initial matrix """
    m = len(string1) + 1 #length of string 1
    n = len(string2) + 1 #length of string 2
    matrix = np.zeros((m, n)) #create a matrix of dimensions m x n with zeros
    for x in range(m): #iterate over range of m
        matrix[x, 0] = x * del_cost #put the values of the iteration in the first column
    for y in range(n): #iterate over range of n
        matrix[0, y] = y * ins_cost #put the values of the iteration in the first row

    """ Fill in the matrix with minimum cost """
    for x in range(1, m): #iterate over range of m skipping the first value [0]
        for y in range(1, n): #iterate over range of n skipping the first value [0]
            sub = sub_cost if string1[x-1] != string2[y-1] else 0 #determine if there was a substitution         
            matrix[x,y] = min( #select the action with minimum cost 
                matrix[x-1, y-1] + sub, #calculate the substitution cost
                matrix[x-1, y] + del_cost, #calculate the deletion cost
                matrix[x, y-1] + ins_cost #calculate the insertion cost
            )
    
    """ Calculate the optimal sequence of operations """
    row = m-1 #create row variable
    column = n-1 #create column variable
    operations = [] #create list for optimal sequence of operations
    while not ((row == 0) and (column == 0)): #find optimal path from cell [row,column] until cell [0,0]
        if (row != 0) & (column != 0): #if we haven't reached the first row or the first column
            #Value of cell to the left is smaller than cell above and diagonal cell """
            if (matrix[row,column-1] + ins_cost < matrix[row-1,column] + del_cost) \
            & (matrix[row,column-1] + ins_cost < matrix[row-1,column-1] + sub_cost): 
                column = column-1 #shift to the left
                operations.append('I') #add Insertion to our list 
            #Value of cell above is smaller than cell to the left and diagonal cell
            elif (matrix[row-1,column] + del_cost < matrix[row,column-1] + ins_cost) \
            & (matrix[row-1,column] + del_cost < matrix[row-1,column-1] + sub_cost): 
                row = row-1 #shift upwards
                operations.append('D') #add Deletion to our list
            #Value of diagonal cell is smaller than cell to the left and cell above
            elif (matrix[row-1,column-1] + sub_cost <= matrix[row,column-1] + ins_cost) \
            & (matrix[row-1,column-1] + sub_cost <= matrix[row-1,column] + del_cost): 
                if matrix[row-1,column-1] == matrix[row,column] - sub_cost: #value of diagonal cell is equal to current cell + substitution cost
                    row = row-1 #shift upwards
                    column = column-1 #and shift to the left 
                    operations.append('S') #add Substitution to our list
                else: #value of diagonal cell is equal to current cell
                    row = row-1 #shift upwards
                    column = column-1 #and shift to the left
                    operations.append('-') #add Blank as no operations were needed
        elif column != 0: #if we have reached to the first row
            column = column-1 #shift to the left
            operations.append('I') #add Insertion to our list
        else: #if we have reached to the first column
            row = row-1 #shift upwards
            operations.append('D') #add Deletion to our list
    operations = list(reversed(operations)) #reverse the order of the list, so that we have from beginning to end
    
    """ Print the results """
    print ('Optimal sequence: ', operations) #print the optimal sequence of operations
    return (int(matrix[m - 1, n - 1])) #return the edit distance

In [8]:
# Print the results from DNA with penalty
st = time()
print('Edit distance: ', edit_distance_penalty(X1, Y1))

# Print time from DNA with penalty
print('Time: %0.5f seconds' % (time() - st))

Optimal sequence:  ['D', 'D', '-', '-', '-', '-', '-', '-', 'S', '-', '-', '-', '-', '-', '-', '-', '-', 'S', 'S', '-', '-', '-', '-', '-', '-', '-', 'D', 'D', '-', '-', '-', '-', '-', '-', '-', 'D', 'S', '-', 'S', '-', '-', '-', '-', '-', '-', '-']
Edit distance:  15
Time: 0.00424 seconds


In [9]:
# Print the results from Proteins with penalty
st = time()
print('Edit distance: ', edit_distance_penalty(X2, Y2))

# Print time from Proteins with penalty
print('Time: %0.5f seconds' % (time() - st))

Optimal sequence:  ['D', 'D', '-', 'D', 'D', 'D', 'D', '-', 'S', '-', 'S', '-', 'S', 'S', 'S', 'S', 'S', 'S', '-', 'S', 'S', '-', 'S', 'S', '-', 'S', '-', 'S', 'S', '-', '-', 'S', '-', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', '-', 'D', 'D', 'D', '-', 'D', 'D', 'D', 'D']
Edit distance:  58
Time: 0.00354 seconds
